# Carbon Emission Prediction Model

In [28]:
from datetime import date
import time
import torch # pip install torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import os
from ipynb.fs.defs.train_benchmark import process_data; # pip install ipynb

## Dataset load and preprocess

In [29]:
# Set dataset locations
selected_output_variable = "NO2"
data_file = "01_Data/02_Imagery/data_and_imagery_test.pkl"

In [30]:
X, y, res, num_channels, m = process_data(data_file, selected_output_variable) # from benchmark code
# reshape and convert datatype
X = torch.from_numpy(X.T).to(torch.float)
y = torch.from_numpy(y.T).to(torch.float)

# split dataset
percent_train = 0.80
num_train = int(percent_train * m)
num_test = m - num_train
print('num training samples: ', num_train)
print('num testing samples: ', num_test)

X_train, X_test = torch.utils.data.random_split(X, [num_train, num_test])
y_train, y_test = torch.utils.data.random_split(y, [num_train, num_test])

# convert from Subset to Tensor type
X_train, X_test = X_train.dataset[X_train.indices], X_test.dataset[X_test.indices]
y_train, y_test = y_train.dataset[y_train.indices], y_test.dataset[y_test.indices]

1431
9
num training samples:  1152
num testing samples:  288


## Define model

In [31]:
class Net(nn.Module):
    """
    Define the neural network: 1 hidden layer with ReLU activation
    """
    def __init__(self, res, num_channels, m):
        """
        Define the network layers
        """
        super(Net,self).__init__()
        self.layer1 = nn.Linear(res*res*num_channels, m) # computes W^T X + b
        
    def forward(self, x):
        """
        Define forward pass
        """
        x = torch.nn.functional.relu(self.layer1(x)) # ReLU activation
        return x

## Train model

In [36]:
def train(x_train, y_train, num_epochs, save=False, model_tag=''):
    """
    Train model on training data. If save=True, trained model is saved in ./03_Trained_Models/NN/model_date_<model_tag>.pt
    Input:
        x data and labels
    Output:
        trained model
    """
    print('Training...')
    t0 = time.time()
    train_losses = []
    optimizer.zero_grad() # clear gradients
    for epoch in range(num_epochs):
        output = net(x_train)
        loss = criterion(output, y_train)
        if (epoch%10 == 0): # save every 10th loss
            train_losses.append(loss)
            print('\tloss at epoch %i = %f' %(epoch, loss))
        loss.backward()
        optimizer.step()

    # save trained model and training details
    if (save):
        train_date = date.today()
        folder_path = os.path.join('03_Trained_Models', 'NN', 'model_%s_%s' %(train_date, model_tag))
        if (not os.path.exists(folder_path)):
            os.mkdir(folder_path)
        
        # save trained model
        path = os.path.join(folder_path, 'model_%s_%s.pt' %(train_date, model_tag))
        torch.save(net.state_dict(), path)
        
        # save training details
        path = os.path.join(folder_path, 'model_%s_%s_training.txt' %(train_date, model_tag))
        with open(path, 'w') as f:
            f.write("Size of x_train: %s" %(str(x_train.shape)))
            f.write("\nSize of y_train: %s\n" %(str(y_train.shape)))
            for i, loss in enumerate(train_losses):
                f.write("\nLoss at epoch %i: %f" %(i*10, loss)) # CHANGE THIS
            f.write("\n\nTotal train time: %fs" %(time.time() - t0))
        
    return net

## Define hyperparameters

In [33]:
num_epochs = 100 # number iterations for gradient descent

## Main

In [37]:
# initialize network
net = Net(res, num_channels, m)
L = len(list(net.parameters())) # number of layers
# print(net)

# initialize loss function and optimizer
criterion = nn.MSELoss() # for regression
optimizer = optim.SGD(net.parameters(), lr=0.05)

# train model and save to ./03_Trained_Models/NN/model_date_<model_tag>.pt
model_tag = '1_hidden_relu'
train(X, y, num_epochs, True, model_tag)

# test model
print('Testing...')
output = net(X_test)
loss = criterion(output, y_test).item()
print('\tTest loss = ', loss)

Training...
	loss at epoch 0 = 745.739380
	loss at epoch 10 = 368.665161
	loss at epoch 20 = 475.862396
	loss at epoch 30 = 668.649353
	loss at epoch 40 = 289.277344
	loss at epoch 50 = 608.011108
	loss at epoch 60 = 577.870361
	loss at epoch 70 = 309.116486
	loss at epoch 80 = 678.367249
	loss at epoch 90 = 443.373505
Testing...
	Test loss =  406.5015869140625
